In [2]:
import dotenv
import instagrapi
import httpx

from config import InstaSettings
from services.insta_service import InstaService
from dto.instagram import *

dotenv.load_dotenv()

True

In [3]:
insta_settings = InstaSettings()
client = instagrapi.Client()

In [4]:
client.login(username=insta_settings.USERNAME, password=insta_settings.PASSWORD)

True

In [5]:
url = "https://www.instagram.com/p/DKXAZ2CtnI3/?utm_source=ig_web_copy_link"
url = InstaService.process_url(url)
url

'https://www.instagram.com/p/DKXAZ2CtnI3/'

In [6]:
media_pk = client.media_pk_from_url(url)
media_info = MyMedia.model_validate(client.media_info(media_pk), from_attributes=True)

In [7]:
media_info.model_dump()

{'pk': 3645384199394193975,
 'id': '3645384199394193975_4020549143',
 'code': 'DKXAZ2CtnI3',
 'taken_at': datetime.datetime(2025, 6, 1, 13, 14, 42, tzinfo=TzInfo(UTC)),
 'media_type': <MediaType.ALBUM: 8>,
 'image_versions2': {'candidates': [{'url': HttpUrl('https://scontent-ams2-1.cdninstagram.com/v/t51.2885-15/503175935_18375291760133144_8156516682850075176_n.jpg?stp=dst-jpg_e15_p360x360_tt6&efg=eyJ2ZW5jb2RlX3RhZyI6IkNBUk9VU0VMX0lURU0uaW1hZ2VfdXJsZ2VuLjcyMHg5MDAuc2RyLmY3NTc2MS5kZWZhdWx0X2NvdmVyX2ZyYW1lIn0&_nc_ht=scontent-ams2-1.cdninstagram.com&_nc_cat=108&_nc_oc=Q6cZ2QHlm1YCZg1GNzDPFICscqdcP81VrJIvzf7BZEru9CXo2C_zRgTGXXM0WaQk7QuOsfY&_nc_ohc=S0EpJDpI1yMQ7kNvwH5YUdi&_nc_gid=AgeT8eNA4_87fftaM-VC9w&edm=ALQROFkBAAAA&ccb=7-5&ig_cache_key=MzY0NTM4MzgwMTEwMjk0Njg0NQ%3D%3D.3-ccb7-5&oh=00_AfPQB8BV1k53ir5SDwbMm9H65RmJWgXH8TxXkpnjePEy6g&oe=68598F0C&_nc_sid=fc8dfb'),
    'height': 450,
    'width': 360}]},
 'product_type': 'carousel_container',
 'thumbnail_url': None,
 'location': None,
 'user':

In [8]:
media_info.extract_media_urls()

['https://scontent-ams2-1.cdninstagram.com/o1/v/t2/f2/m367/AQMczoaR4Q_q07EMcGKzV1-tu-l5nFfc2BPK4Yt-fLwmU2wFBy-aoRDejjC26e6Y6fI6g3ru9E6SQMy_mZx__HPwLoUEwj1Vif2nIwU.mp4?_nc_cat=106&_nc_sid=5e9851&_nc_ht=scontent-ams2-1.cdninstagram.com&_nc_ohc=3Mm60WeAszwQ7kNvwGmWyC1&efg=eyJ2ZW5jb2RlX3RhZyI6Inhwdl9wcm9ncmVzc2l2ZS5JTlNUQUdSQU0uQ0FST1VTRUxfSVRFTS5DMy4xMDgwLmRhc2hfYmFzZWxpbmVfMTA4MHBfdjEiLCJ4cHZfYXNzZXRfaWQiOjEwMzE2OTA2OTUzMDEwOTgsInZpX3VzZWNhc2VfaWQiOjEwMTQ2LCJkdXJhdGlvbl9zIjoxNiwidXJsZ2VuX3NvdXJjZSI6Ind3dyJ9&ccb=17-1&vs=d9d87940b406943d&_nc_vs=HBksFQIYQGlnX2VwaGVtZXJhbC9BMDQzRkU2QzhFOTkzRkRFNUY2NDNBRDRGNzk3OTM4MF92aWRlb19kYXNoaW5pdC5tcDQVAALIARIAFQIYOnBhc3N0aHJvdWdoX2V2ZXJzdG9yZS9HUFRJOVIzSHRNVU5WQmdqQUJ2QlpBSERxdzBPYmtZTEFBQUYVAgLIARIAKAAYABsCiAd1c2Vfb2lsATEScHJvZ3Jlc3NpdmVfcmVjaXBlATEVAAAm1N_K1qaU1QMVAigCQzMsF0Aw1T987ZFoGBZkYXNoX2Jhc2VsaW5lXzEwODBwX3YxEQB17gdlxJ4BAA&_nc_zt=28&oh=00_AfOHv-l72Cdt5ta9L0e40egYOVrZIYw_Kytey6ea-MQjsw&oe=685979C7',
 'https://scontent-ams4-1.cdninstagram.com/o1

In [10]:
result = 0
for url in media_info.extract_media_urls():
    response = httpx.get(url)
    assert response.is_success
    result += len(response.content)

In [13]:
result / (1024 ** 2)

14.41268539428711